In [1]:
! pip install beautifulsoup4
! requests
! re
! pip install selenium
!apt install chromium-chromedriver

/bin/bash: requests: command not found
/bin/bash: re: command not found
     |████████████████████████████████| 968 kB 6.8 MB/s 
     |████████████████████████████████| 138 kB 14.6 MB/s 
     |████████████████████████████████| 359 kB 34.0 MB/s 
     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 4.0 MB 31.9 MB/s 
     |████████████████████████████████| 58 kB 5.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Reading package lists... Done
Building dep

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

검색키워드 입력

In [4]:
keyword = input('검색키워드: ')
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_0_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

time.sleep(2)

검색키워드: 키보드


크롤링 일단 한페이지부터

In [5]:
base_url = 'https://www.coupang.com/np/search?component=&q={}&channel=user&page={}'
url = base_url.format(keyword,1)

driver.execute_script('window.open("https://google.com");')  #구글 창 새 탭으로 열기
time.sleep(1)

driver.switch_to.window(driver.window_handles[-1])  #새로 연 탭으로 이동
driver.get(url)
time.sleep(3)
driver.close()  #링크 이동 후 탭 닫기
driver.switch_to.window(driver.window_handles[-1])  #다시 이전 창(탭)으로 

res = requests.get(url, headers = headers,timeout=30)
#res.raise_for_status()
soup = BeautifulSoup(res.content, 'html.parser')

ReadTimeout: ignored

In [ ]:
lastpage = soup.select_one('a.btn-last').text.strip()
lastpage




In [ ]:
for page in range(1, int(lastpage)+1):
  url = base_url.format(keyword, page)
  time.sleep(2)
  driver.execute_script('window.open("https://google.com");')  #구글 창 새 탭으로 열기
  time.sleep(1)
  driver.switch_to.window(driver.window_handles[-1])  #새로 연 탭으로 이동
  driver.get(url)
  time.sleep(3)
  driver.close()  #링크 이동 후 탭 닫기
  driver.switch_to.window(driver.window_handles[-1])  #다시 이전 창(탭)으로 

  res = requests.get(url, headers = headers, timeout=30)

  product_count_list = []
  product_price_list = []
  product_review_list = []


# 페이지당 상품개수 추출하기
  product_name = soup.select('dl.search-product-wrap')
  product_count = len(product_name) #한페이지당 상품 개수
  product_count_list.append(product_count) 

# 상품 가격 추출하기
  product_price = soup.select('dl.search-product-wrap > dd.descriptions > div.descriptions-inner > div.price-area > div.price-wrap > div.price > em > strong')
  price_count = len(product_price)
  sum_price = 0
  for k in range(price_count):
    price_list = int(product_price[k].text.replace(',',""))
    sum_price += price_list
    avg_price = sum_price/price_count
  product_price_list.append(avg_price)
  
# 상품 리뷰 평점 추출하기
  product_review = soup.select('dl.search-product-wrap > dd.descriptions > div.descriptions-inner > div.other-info > div > span.star')
  review_count=len(product_review)
  sum_review = 0
  for k in range(review_count):
    review_list = int(product_review[k].text.replace('.',""))
    sum_review += review_list
    avg_review = (sum_review/10)/review_count
  product_review_list.append(avg_review)
  driver.close()


In [ ]:
product_count_list

In [ ]:
product_price_list

In [ ]:
product_review_list

In [ ]:
product_list = product_count_list+product_price_list+product_review_list
product_list



크롤링


> 베스트셀러, 추천상품 제외하고 온전한 상품만


In [ ]:
product_list = []
# 페이지당 상품개수 추출하기
product_name = soup.select('dl.search-product-wrap')
product_count = len(product_name) #한페이지당 상품 개수
product_list.append(product_count) 

# 상품 가격 추출하기
product_price = soup.select('dl.search-product-wrap > dd.descriptions > div.descriptions-inner > div.price-area > div.price-wrap > div.price > em > strong')
price_count = len(product_price)
sum_price = 0
for k in range(price_count):
  price_list = int(product_price[k].text.replace(',',""))
  sum_price += price_list
  avg_price = sum_price/price_count
product_list.append(avg_price)
  
# 상품 리뷰 평점 추출하기
product_review = soup.select('dl.search-product-wrap > dd.descriptions > div.descriptions-inner > div.other-info > div > span.star')
review_count=len(product_review)
sum_review = 0
for k in range(review_count):
  review_list = int(product_review[k].text.replace('.',""))
  sum_review += review_list
  avg_review = (sum_review/10)/review_count
product_list.append(avg_review)

상품 개수 및 리뷰 평점 평균 전처리


In [ ]:
product_list

상품검색 마지막 페이지 수

In [ ]:
final_df = pd.DataFrame(product_list)
final_df = final_df.transpose()
final_df.columns = ['경쟁제품 개수','평균 가격','평균 별점']
final_df.head()